In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle competitions download -c induction-task

 97% 194M/201M [00:01<00:00, 196MB/s]
100% 201M/201M [00:01<00:00, 182MB/s]


In [4]:
import zipfile
zip_f = zipfile.ZipFile('/content/induction-task.zip', 'r')
zip_f.extractall('/content')
zip_f.close()

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/Data/Train"
new_size = (256,256)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=new_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=new_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 641 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [7]:

import os
import cv2

image_folder = "/content/Data/Train/Real"
output_folder = "/content/Real_resized"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, new_size)
        cv2.imwrite(os.path.join(output_folder, filename), img_resized)

In [8]:
image_folder = "/content/Data/Train/AI"
output_folder = "/content/AI_resized"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, new_size)
        cv2.imwrite(os.path.join(output_folder, filename), img_resized)

In [9]:
import numpy as np
from PIL import Image

image_folder = "/content/Real_resized"

image_array_list = []

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img_array = np.array(img)
        image_array_list.append(img_array)

real_array = np.stack(image_array_list)

print(f"Final Shape of Array: {real_array.shape}")

Final Shape of Array: (400, 512, 512, 3)


In [10]:
image_folder = "/content/AI_resized"

image_array_list = []

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder, filename)
        img_array = np.array(img)
        image_array_list.append(img_array)

ai_array = np.stack(image_array_list)

print(f"Final Shape of Array: {ai_array.shape}")

Final Shape of Array: (401, 512, 512, 3)


In [11]:

label_ai = np.zeros(len(ai_array))
label_real = np.ones(len(real_array))

X = np.concatenate((ai_array, real_array), axis=0)
Y = np.concatenate((label_ai, label_real), axis=0)

print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {Y.shape}")

Shape of X: (801, 512, 512, 3)
Shape of Y: (801,)


In [12]:
X=X/255.0

In [44]:

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(256,256, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(900, activation='relu'),
    Dense(150, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [45]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [47]:
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=12,
    batch_size=32  # Reduced batch size
)


Epoch 1/12


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt

In [ ]:
model.evaluate(validation_generator)

In [ ]:
import cv2
image_folder = "/content/Data/Test"
output_folder = "/content/Test_resized"
os.makedirs(output_folder, exist_ok=True)
new_size=(256,256)
a=0
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)

        # Check if image was loaded successfully
        if img is None:
            print(f"Error loading image: {img_path}")
            continue  # Skip to the next image

        img_resized = cv2.resize(img, new_size)
        cv2.imwrite(os.path.join(output_folder, filename), img_resized)
        a+=1

print("All images have been reshaped and saved.")
print(a)

In [ ]:
image_folder = "/content/Test_resized"

def natural_sort_key(filename):
    import re
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

image_filenames = sorted(
    [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png", ".jpeg"))],
    key=natural_sort_key
)

image_array_list = []
image_size=(256,256)
for filename in image_filenames:
    img_path = os.path.join(image_folder, filename)
    img = Image.open(img_path).resize(image_size)
    img_array = np.array(img)
    image_array_list.append(img_array)

test_array = np.stack(image_array_list)

print(f"Final array shape: {test_array.shape}")


In [ ]:
test_array=test_array/255.0
prediction = model.predict(test_array)

In [ ]:
import csv

image_ids = [f'image_{i+1}' for i in range(200)]

labels = ['Real' if pred > 0.5 else 'AI' for pred in prediction]

if len(image_ids) != len(labels):
    print("Error: Mismatch between number of image IDs and predictions.")
else:
    with open('submission.csv', mode='w', newline='') as file:
        writer = csv.writer(file)

        writer.writerow(['Id', 'Label'])

        for img_id, label in zip(image_ids, labels):
            writer.writerow([img_id, label])

    print("Submission file created: submission.csv")